<a href="https://colab.research.google.com/github/coltonro/Pytorch-Train-Model/blob/main/GameCameraPhotoAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Custom YOLOv8n Training for Animal Detection

## Setup and Installation

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install ultralytics
!pip install xml-to-yolo

# Import necessary libraries
import os
import glob
import shutil
import random
import torch
from xml.etree import ElementTree as ET
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

## Data Preparation

# Set paths
data_dir = '/content/drive/MyDrive/GameCameraPhotoAI_Training/master_training_folder_05-04-24'  # Update this path to your folder containing both images and XML files
output_dir = os.path.join(data_dir, 'yolo_dataset')
best_model_path = os.path.join(output_dir, 'best_model.pt')

# Create YOLO dataset structure
os.makedirs(os.path.join(output_dir, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images', 'test'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'test'), exist_ok=True)

# Define classes
classes = ['blue_jay', 'house_sparrow', 'northern_cardinal', 'white_winged_dove', 'black_vulture',
           'lesser_goldfinch', 'european_starling', 'golden_fronted_woodpecker', 'northern_mockingbird',
           'black_crested_titmouse', 'yellow_billed_cuckoo', 'carolina_wren', 'cat', 'deer', 'squirrel',
           'person', 'raccoon', 'skunk', 'opossum', 'fox']

# Function to convert XML to YOLO format
def convert_xml_to_yolo(xml_file, output_dir, image_width=1138, image_height=640):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_lines = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in classes:
            continue
        class_id = classes.index(class_name)

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        # Convert to YOLO format
        x_center = (xmin + xmax) / (2 * image_width)
        y_center = (ymin + ymax) / (2 * image_height)
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        yolo_lines.append(f"{class_id} {x_center} {y_center} {width} {height}")

    # Write YOLO format to file
    base_name = os.path.splitext(os.path.basename(xml_file))[0]
    with open(os.path.join(output_dir, f"{base_name}.txt"), 'w') as f:
        f.write("\n".join(yolo_lines))

# Convert annotations and split dataset
image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]
random.shuffle(image_files)

train_files, test_val_files = train_test_split(image_files, test_size=0.4, random_state=42)
val_files, test_files = train_test_split(test_val_files, test_size=0.5, random_state=42)

for img_file in image_files:
    xml_file = os.path.join(data_dir, img_file.replace('.jpg', '.xml'))
    if os.path.exists(xml_file):
        if img_file in train_files:
            subset = 'train'
        elif img_file in val_files:
            subset = 'val'
        else:
            subset = 'test'

        # Copy image
        shutil.copy(os.path.join(data_dir, img_file),
                    os.path.join(output_dir, 'images', subset, img_file))

        # Convert and save annotation
        convert_xml_to_yolo(xml_file, os.path.join(output_dir, 'labels', subset))

# Create dataset.yaml file
yaml_content = f"""
train: {os.path.join(output_dir, 'images', 'train')}
val: {os.path.join(output_dir, 'images', 'val')}
test: {os.path.join(output_dir, 'images', 'test')}

nc: {len(classes)}
names: {classes}
"""

with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as f:
    f.write(yaml_content)

## Model Training

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU being used: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data=os.path.join(output_dir, 'dataset.yaml'),
    epochs=100,
    imgsz=640,
    patience=20,
    batch=64,
    lr0=0.01,
    device=0,
    project=output_dir,
    name='best_colab_9_26_24',
    save=True,  # Save best model after each epoch
    save_period=1,  # Save checkpoint every epoch
    exist_ok=True  # Overwrite existing files
)

# After training, copy the best model to a specific location
best_model = max(results.save_dir.glob('*.pt'), key=os.path.getctime)
os.replace(best_model, best_model_path)
print(f"Best model saved to: {best_model_path}")

# Evaluate the model
results = model.val()

print(f"mAP@0.5: {results.box.map50}")
print(f"mAP@0.5:0.95: {results.box.map}")

## Inference

# Perform inference on a test image
test_image = os.path.join(output_dir, 'images', 'test', random.choice(test_files))
results = model(test_image)

# Display results
results[0].plot()

# Save the trained model
model.save('best_colab_9_26_24.pt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Could not find a version that satisfies the requirement xml-to-yolo (from versions: none)
ERROR: No matching distribution found for xml-to-yolo
Current working directory: /content
YOLOv8 default save directory not found.
Model object not found in memory.
No .pt files found.


KeyboardInterrupt: 